# STEP 2: Autoencoder per Anomaly Detection - UN MODELLO PER OGNI CONNETTORE

Questo notebook addestra **9 modelli separati**, uno per ogni connettore (conn1, conn2, ..., conn9).

Ogni modello avrà il suo threshold specifico calcolato solo sui dati OK del rispettivo connettore.

## Setup

In [ ]:
# Setup: Clona repository GitHub e monta Google Drive per i dati
import os
from pathlib import Path

# Opzione 1: Clona da GitHub (consigliato per sviluppo)
# Sostituisci con il tuo repository URL
GITHUB_REPO = "https://github.com/Giovanni000/Project-Work.git"  # ⚠️ MODIFICA QUESTO!
REPO_DIR = "/content/project"

# Clona repository (se non esiste già)
if not Path(REPO_DIR).exists():
    !git clone {GITHUB_REPO} {REPO_DIR}
else:
    os.chdir(REPO_DIR)
    !git pull

# Cambia directory al repository
os.chdir(REPO_DIR)
# Se il clone crea una sottocartella, entra dentro
subdirs = [d for d in Path(REPO_DIR).iterdir() if d.is_dir() and not d.name.startswith('.')]
if len(subdirs) == 1:
    os.chdir(subdirs[0])

print(f"Repository directory: {os.getcwd()}")

# Opzione 2: Monta Google Drive solo per i dati (immagini)
from google.colab import drive
drive.mount('/content/drive')

# Path ai dati su Drive
DATA_ROOT = Path("/content/drive/MyDrive/Project Work/Data")
print(f"Data directory: {DATA_ROOT}")

# ⚠️ IMPORTANTE: Le immagini su Drive sono LENTE da caricare durante il training!
# Se il training è troppo lento, considera di copiare le immagini in locale prima

# Import necessari
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from pathlib import Path
import numpy as np
from tqdm import tqdm

# Seed per riproducibilità
torch.manual_seed(42)
np.random.seed(42)

# Verifica device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")
    if "T4" in gpu_name:
        print("✅ Tesla T4 rilevata - Parametri ottimizzati per questa GPU")


## Dataset Class (solo OK, filtrato per connettore)


In [ ]:
class AEDatasetPerConnector(Dataset):
    """
    Dataset PyTorch per autoencoder di un singolo connettore.
    Contiene solo immagini OK del connettore specificato.
    """
    
    def __init__(self, csv_path, connector_name, transform=None):
        """
        Args:
            csv_path: Path al CSV con colonne 'image_path', 'label', 'connector_name'
            connector_name: Nome del connettore (es. 'conn1', 'conn2', ...)
            transform: Trasformazioni da applicare alle immagini
        """
        df = pd.read_csv(csv_path)
        # Filtra solo OK del connettore specificato
        self.df = df[(df['label'] == 'OK') & (df['connector_name'] == connector_name)].copy().reset_index(drop=True)
        self.transform = transform
        
        print(f"Dataset AE per {connector_name}: {len(self.df)} immagini OK")
        if len(self.df) == 0:
            raise ValueError(f"Nessuna immagine OK trovata per {connector_name}!")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        
        # Carica immagine (ottimizzato: evita lazy loading)
        try:
            image = Image.open(image_path).convert('RGB')
            image.load()  # Forza caricamento completo
        except Exception as e:
            print(f"Errore caricamento {image_path}: {e}")
            image = Image.new('RGB', (128, 128), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        return image


## Modello Autoencoder Convoluzionale (stesso di prima)


In [ ]:
class ConvAE(nn.Module):
    """
    Autoencoder convoluzionale per anomaly detection.
    
    Encoder: 3 conv2d con stride=2 (3→16→32→64 canali)
    Decoder: 3 convtranspose2d simmetriche (64→32→16→3 canali)
    """
    
    def __init__(self):
        super(ConvAE, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            # 128x128x3 -> 64x64x16
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            # 64x64x16 -> 32x32x32
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            # 32x32x32 -> 16x16x64
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            # 16x16x64 -> 32x32x32
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            # 32x32x32 -> 64x64x16
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            # 64x64x16 -> 128x128x3
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  # Output in [0, 1]
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


## Funzione Training (per un singolo connettore)


In [ ]:
def train_autoencoder_per_connector(connector_name, csv_path="data/dataset.csv",
                                     batch_size=128,
                                     num_epochs=30,
                                     learning_rate=0.001,
                                     device=None):
    """
    Addestra un autoencoder per un singolo connettore.
    
    Args:
        connector_name: Nome del connettore (es. 'conn1')
        csv_path: Path al CSV del dataset
        batch_size: Dimensione del batch
        num_epochs: Numero di epoche
        learning_rate: Learning rate
        device: Device (cuda/cpu)
    
    Returns:
        model: Modello addestrato
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print(f"\n{'='*60}")
    print(f"Training autoencoder per {connector_name}")
    print(f"{'='*60}")
    
    # Trasformazioni
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])
    
    # Dataset (solo OK del connettore specificato)
    dataset = AEDatasetPerConnector(csv_path, connector_name, transform=transform)
    
    # DataLoader (ottimizzato per Colab)
    train_loader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2,  # 2 per Tesla T4
        pin_memory=True if device.type == 'cuda' else False,
        prefetch_factor=2,
        persistent_workers=False
    )
    
    # Modello
    model = ConvAE().to(device)
    
    # Loss e Optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = images.to(device)
            
            optimizer.zero_grad()
            reconstructed = model(images)
            loss = criterion(reconstructed, images)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}")
    
    # Salva modello
    models_dir = Path("models")
    models_dir.mkdir(exist_ok=True)
    model_path = models_dir / f"ae_conv_{connector_name}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"\n✅ Modello salvato in: {model_path}")
    
    return model


## Funzione Calcolo Threshold (per un singolo connettore)


In [ ]:
def calculate_threshold_per_connector(model, connector_name, csv_path="data/dataset.csv", device=None):
    """
    Calcola il threshold per anomaly detection per un singolo connettore.
    
    Threshold = mu + 3*sigma, dove mu e sigma sono media e std degli errori
    di ricostruzione su tutte le immagini OK del connettore specificato.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print(f"\nCalcolo threshold per {connector_name}...")
    
    # Trasformazioni
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])
    
    # Dataset (solo OK del connettore specificato)
    dataset = AEDatasetPerConnector(csv_path, connector_name, transform=transform)
    loader = DataLoader(
        dataset, 
        batch_size=32, 
        shuffle=False, 
        num_workers=2,
        pin_memory=True if device.type == 'cuda' else False,
        persistent_workers=False
    )
    
    model.eval()
    errors = []
    criterion = nn.MSELoss(reduction='none')
    
    with torch.no_grad():
        for images in tqdm(loader, desc="Calcolo errori"):
            images = images.to(device)
            reconstructed = model(images)
            
            batch_errors = criterion(reconstructed, images)
            batch_errors = batch_errors.mean(dim=(1, 2, 3))
            
            errors.extend(batch_errors.cpu().numpy())
    
    errors = np.array(errors)
    mu = np.mean(errors)
    sigma = np.std(errors)
    
    # ⚙️ SCEGLI IL METODO PER IL THRESHOLD:
    # Opzione 1: mu + 3*sigma (più conservativo, meno falsi positivi, ma più falsi negativi)
    # Opzione 2: mu + 2*sigma (più sensibile, rileva più KO, ma più falsi positivi)
    # Opzione 3: mu + 2.5*sigma (compromesso)
    
    # Se non rilevi KO, prova a ridurre il moltiplicatore (es. 2 o 2.5 invece di 3)
    SIGMA_MULTIPLIER = 2.5  # ⚠️ MODIFICA QUESTO se non rilevi KO (prova 2.0 o 2.5)
    threshold = mu + SIGMA_MULTIPLIER * sigma
    
    print(f"\nStatistiche errori per {connector_name}:")
    print(f"  Media (mu): {mu:.6f}")
    print(f"  Std (sigma): {sigma:.6f}")
    print(f"  Min: {errors.min():.6f}")
    print(f"  Max: {errors.max():.6f}")
    print(f"\nThreshold (mu + 3*sigma): {threshold:.6f}")
    
    # Salva threshold
    models_dir = Path("models")
    models_dir.mkdir(exist_ok=True)
    threshold_path = models_dir / f"ae_threshold_{connector_name}.npy"
    np.save(threshold_path, threshold)
    print(f"  Threshold salvato in: {threshold_path}")
    
    return threshold


## Addestra Tutti i 9 Modelli


In [ ]:
# Lista dei 9 connettori
connectors = [f"conn{i}" for i in range(1, 10)]

# Verifica che il CSV esista
csv_path = Path("data/dataset.csv")
if not csv_path.exists():
    print("⚠️  data/dataset.csv non trovato! Esegui prima training_pipeline.ipynb")
    raise FileNotFoundError("data/dataset.csv non trovato")

# ⚙️ CONFIGURAZIONE TRAINING
# Se la loss sta ancora scendendo, aumenta le epoche
# Nota: Se la loss scende molto lentamente (< 0.00002 per epoca), i guadagni sono minimi
# Per anomaly detection, una loss < 0.002 è già eccellente
NUM_EPOCHS = 100  # Aumentato a 100 (puoi aumentare a 120-150 se la loss scende ancora lentamente)
BATCH_SIZE = 128  # Ottimizzato per Tesla T4
LEARNING_RATE = 0.001

print(f"⚙️  Configurazione:")
print(f"   Epoche: {NUM_EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")

# Addestra un modello per ogni connettore
models = {}
thresholds = {}

for connector_name in connectors:
    try:
        # Training
        model = train_autoencoder_per_connector(
            connector_name=connector_name,
            csv_path=str(csv_path),
            batch_size=BATCH_SIZE,
            num_epochs=NUM_EPOCHS,
            learning_rate=LEARNING_RATE,
            device=device
        )
        models[connector_name] = model
        
        # Calcola threshold
        threshold = calculate_threshold_per_connector(
            model=model,
            connector_name=connector_name,
            csv_path=str(csv_path),
            device=device
        )
        thresholds[connector_name] = threshold
        
    except Exception as e:
        print(f"❌ Errore per {connector_name}: {e}")
        continue

print(f"\n{'='*60}")
print(f"✅ Training completato per {len(models)} connettori")
print(f"{'='*60}")
print("\nRiepilogo thresholds:")
for conn, thresh in thresholds.items():
    print(f"  {conn}: {thresh:.6f}")


## Funzione Helper per Caricare Modello e Threshold


In [ ]:
def load_ae_and_threshold_per_connector(connector_name, device=None):
    """
    Carica l'autoencoder addestrato e il threshold per un singolo connettore.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Carica modello
    model = ConvAE().to(device)
    model_path = Path(f"models/ae_conv_{connector_name}.pth")
    
    if not model_path.exists():
        raise FileNotFoundError(f"Modello non trovato: {model_path}")
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    # Carica threshold
    threshold_path = Path(f"models/ae_threshold_{connector_name}.npy")
    if not threshold_path.exists():
        raise FileNotFoundError(f"Threshold non trovato: {threshold_path}")
    
    threshold = np.load(threshold_path)
    
    return model, threshold
